In [7]:
import pandas as pd
import numpy as np
import random,re
import json
from datasets import load_dataset
from utils import *
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
pd.set_option('display.max_colwidth', None)

In [8]:
dataset = load_dataset('meta-math/MetaMathQA')['train']
dataset = [x for x in dataset if x['type'][:3] == 'GSM']

with open('/cluster/project/sachan/liuron/Thesis/experiments/SVAMP.json','r') as json_file:
    dataset_svamp = json.load(json_file)
dataset_gsm = load_dataset('gsm8k','main')['test']
dataset_asdiv = ET.parse('/cluster/project/sachan/liuron/Thesis/experiments/ASDiv.xml')


In [9]:
# count = 0
# equations = [x['Equation'] for x in dataset_svamp]
# for equation in equations:
#     temp = 0
#     for char in equation:
#         if char in ['+','-','*','/']:
#             temp += 1
#     if temp > 1:
#         count += 1
# print(count)

In [10]:
def xml_to_dict(element):
    # If the element has no children, return its text content
    if len(element) == 0:
        return element.text

    # Otherwise, create a dictionary for the element
    result = {}
    for child in element:
        child_result = xml_to_dict(child)
        if child.tag not in result:
            result[child.tag] = child_result
        else:
            # If the tag already exists, make it a list of results
            if not isinstance(result[child.tag], list):
                result[child.tag] = [result[child.tag]]
            result[child.tag].append(child_result)
    
    # Include element attributes if they exist
    if element.attrib:
        result['@attributes'] = element.attrib

    return result

# Convert the root element to a dictionary
root = dataset_asdiv.getroot()
xml_dict = {root.tag: xml_to_dict(root)}
dataset_asdiv = xml_dict['Machine-Reading-Corpus-File']['ProblemSet']['Problem']

In [11]:
count = 0
equations = [x['Formula'] for x in dataset_asdiv]
for equation in equations:
    temp = 0
    for char in equation:
        if char in ['+','-','*','/']:
            temp += 1
    if temp > 0:
        # print(equation)
        count += 1
print(count)

2054


In [12]:
def remove_duplicates(dataset):
    questions = [x['query'] for x in dataset]
    answers = [x['response'] for x in dataset]
    a = {}
    for i in range(len(questions)):
        if 'sqrt' not in answers[i]:
            a[questions[i]] = answers[i]
    print(len(a))
    res = {'query':[],
           'response':[]}
    for key, value in a.items():
        res['query'].append(key)
        res['response'].append(value)
    return res

#extract prediction
def extract_pred(sample):
    res = ''
    pattern = r'[$]?[-+]?\d+(?:\.\d+)?(?:,\d+)*[$]?'
    matches = re.findall(pattern, sample)
    if matches != []:
        res = float(matches[-1].replace(",", "").replace(" ", "").replace("\n", "").replace("$", "").replace("x", ""))
    return res


unique_data = remove_duplicates(dataset)

112234


In [13]:
example_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

Instruction:
Solve the given math problem step by step. Put your final answer after 'Final answer: '.

Input:
{}

Response:
{}"""

def create_train_data(data):
    problems = data['query']
    solutions = data['response']
    solutions = [x.split('####')[0] + 'Final Answer: ' + x.split('The answer is: ')[1] for x in solutions]
    # official_answer = [extract_pred(x) for x in solutions]
    texts = [example_prompt.format(problems[i],solutions[i]) + '<eos>' for i in range(len(problems))]
    for i in range(10):
        print(texts[i])
        print()
    random.shuffle(texts)
    split_point = int(len(texts) * 0.9)
    train = texts[:split_point]
    val = texts[split_point:]
    dict_train = {'text':train}
    dict_val = {'text':val}
    with open('data/train.json', "w") as json_file:
        json.dump(dict_train, json_file)
    with open('data/val.json', "w") as json_file:
        json.dump(dict_val, json_file)

def create_test_data(data,name):
    if name == 'gsm':
        questions = data['question']
        answers = [extract_pred(data['answer'][i]) for i in range(len(data['answer']))]
    if name == 'svamp':
        questions = [x['Body'] + '' + x['Question'] for x in data]
        answers = [x['Answer'] for x in data]
    if name == 'asdiv':
        questions = [x['Body'] + '' + x['Question'] for x in data]
        answers = [x['Answer'].split(' ')[0] for x in data]
        filtered_questions = []
        filtered_answers = []
        for i in range(len(answers)):
            try:
                ans = float(answers[i])
                filtered_questions.append(questions[i])
                filtered_answers.append(answers[i])
            except:
                continue
        questions = filtered_questions
        answers = filtered_answers
    texts = [example_prompt.format(questions[i],'') for i in range(len(questions))]
    for i in range(5):
        print(texts[i])
        print()
    dict_test = {'text':texts,
                'correct-answer':answers}
    with open(f'data/test_{name}.json', "w") as json_file:
        json.dump(dict_test, json_file)
    
create_train_data(unique_data)
create_test_data(dataset_gsm,'gsm')
create_test_data(dataset_svamp,'svamp')
create_test_data(dataset_asdiv,'asdiv')

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

Instruction:
Solve the given math problem step by step. Put your final answer after 'Final answer: '.

Input:
What is the total cost of purchasing equipment for all sixteen players on the football team, considering that each player requires a $25 jersey, a $15.20 pair of shorts, and a pair of socks priced at $6.80?

Response:
Each player needs a $25 jersey, a $15.20 pair of shorts, and a pair of socks priced at $6.80.
The total cost for each player is $25 + $15.20 + $6.80 = $47.
Since there are 16 players on the team, the total cost for all the equipment is $47 x 16 = $752.
Final Answer: 752<eos>

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

Instruction:
Solve the given math problem step by step. Put your final answer